In [1]:
import pandas as pd
import duckdb

In [ ]:
# Connect to (or create) a DuckDB database file. Ours is hosted on google drive
# you must wait for the file to download if you want to use it (18GB approx)
con = duckdb.connect("G:/My Drive/Studies/DIW/duckdb_stuff/gender_db.duckdb")

In [ ]:
# you can download the db into the directory of this notebook to run the code
# to use local db use:
con = duckdb.connect("gender_db.duckdb")

In [6]:
# Show all tables in db
con.query(""" SHOW TABLES; """)

┌─────────────────────────┐
│          name           │
│         varchar         │
├─────────────────────────┤
│ big_table_eu            │
│ big_table_us            │
│ gender_stats_by_term_eu │
│ gender_stats_by_term_us │
│ gender_stats_by_year_eu │
│ gender_stats_by_year_us │
│ sample_for_labeling     │
└─────────────────────────┘

In [ ]:
# TO REPRODUCE THE CSV INGESTION: USE ONLY IF YOU WANT TO RECREATE DATABASE FROM SCRATCH
# Our raw data is hosted on 248 csv files that have been preprocessed with a seperate python script
# to recreate our database from scratch we can run the code bellow to create the initial tables and validate the data

# concatenate data from all csvs on google drive to two tables in database
con.execute("""
    CREATE OR REPLACE TABLE big_table_eu AS
    SELECT * FROM read_csv_auto("G:/My Drive/Studies/DIW/openalex_eu_final_labeledupd/*.csv", quote='"')
""")

con.execute("""
    CREATE OR REPLACE TABLE big_table_us AS
    SELECT * FROM read_csv_auto("G:/My Drive/Studies/DIW/openalex_us_final_labeledupd/*.csv", quote='"')
""")

# validate row and column counts and that column names have been preserved
# total row count
print(con.execute("SELECT COUNT(*) FROM big_table_eu").fetchone())
print(con.execute("SELECT COUNT(*) FROM big_table_us").fetchone())

# column names
columns = con.execute("PRAGMA table_info('big_table_eu');").fetchall()
print(columns)
columns = con.execute("PRAGMA table_info('big_table_us');").fetchall()
print(columns)

In [ ]:
# create variables for iterative table creation
location = ["eu", "us"]
sciences = ['physical', 'health', 'social', 'life' ]

In [ ]:
# create or replace tables to split data into scientific fields and aggregate by both year and presidential term
for loc in location:
  for science in sciences:
        # aggregate gender counts by year
        con.query(f"""
      CREATE OR REPLACE TABLE gender_year_{science}_sciences_{loc} AS

      SELECT 
          publication_year,
          SUM(CASE WHEN primary_author_gender = 'male' THEN 1 ELSE 0 END) as male,
          SUM(CASE WHEN primary_author_gender = 'female' THEN 1 ELSE 0 END) as female,
          SUM(CASE WHEN primary_author_gender = 'unknown' THEN 1 ELSE 0 END) as uk,
      FROM  (

          SELECT * FROM big_table_{loc}       
          WHERE REGEXP_MATCHES(raw_topic_domains, '.*(?i){science} (?i)science(s)?.*')
          OR
          REGEXP_MATCHES(raw_concept_domains, '.*(?i){science} (?i)science(s)?.*') 
          )

      GROUP BY publication_year
      ORDER BY publication_year
    
      """)
        # aggregates gender counts by persidential terms 
        con.query(f""" 
      CREATE OR REPLACE TABLE gender_term_{science}_sciences_{loc} AS       
      
      WITH my_data AS (
      SELECT * FROM gender_year_{science}_sciences_{loc}
      ),
    
      presidential_terms AS (
            SELECT *
            FROM (
                VALUES
                ('Ronald Reagan', 1981, 1985),    -- 1981 through 1988
                ('Ronald Reagan', 1985, 1989),
                ('George H.W. Bush', 1989, 1993), -- 1989 through 1992
                ('Bill Clinton', 1993, 1997),     -- 1993 through 2000
                ('Bill Clinton', 1997, 2001),
                ('George W. Bush', 2001, 2005),   -- 2001 through 2008
                ('George W. Bush', 2005, 2009),
                ('Barack Obama', 2009, 2013),
                ('Barack Obama', 2013, 2017),     -- 2009 through 2016
                ('Donald Trump', 2017, 2021),     -- 2017 through 2020
                ('Joe Biden', 2021, 2025)        -- 2021 through 2024
            )  AS t(president, start_year, end_year_exclusive)
        )

      SELECT
          t.president,
          t.start_year as start_of_term,
          t.end_year_exclusive as end_of_term,
          SUM(d.male) AS total_male_count,
          SUM(d.female) AS total_female_count,
          SUM(d.uk) AS total_uk_count
      FROM
          my_data d
      JOIN
          presidential_terms t
          ON d.publication_year >= t.start_year AND d.publication_year < t.end_year_exclusive
      GROUP BY
          t.president,
          t.start_year,
          t.end_year_exclusive
      ORDER BY
          MIN(t.start_year);

  """)

In [13]:
con.query(""" show tables """)

┌──────────────────────────────────┐
│               name               │
│             varchar              │
├──────────────────────────────────┤
│ big_table_eu                     │
│ big_table_us                     │
│ gender_term_health_sciences_eu   │
│ gender_term_health_sciences_us   │
│ gender_term_life_sciences_eu     │
│ gender_term_life_sciences_us     │
│ gender_term_physical_sciences_eu │
│ gender_term_physical_sciences_us │
│ gender_term_social_sciences_eu   │
│ gender_term_social_sciences_us   │
│ gender_year_health_sciences_eu   │
│ gender_year_health_sciences_us   │
│ gender_year_life_sciences_eu     │
│ gender_year_life_sciences_us     │
│ gender_year_physical_sciences_eu │
│ gender_year_physical_sciences_us │
│ gender_year_social_sciences_eu   │
│ gender_year_social_sciences_us   │
│ sample_for_labeling              │
├──────────────────────────────────┤
│             19 rows              │
└──────────────────────────────────┘

In [ ]:
# extract a random sample from all data points for validation of automatic gender labels
con.query("""
        CREATE OR REPLACE TABLE sample_for_labeling AS
          
        SELECT * FROM (
          
        SELECT primary_author_name, primary_author_gender FROM big_table_eu
          
        UNION ALL
          
        SELECT primary_author_name, primary_author_gender FROM big_table_us
          
        ) USING SAMPLE 100 """)

# write sample to dataframe
sample = con.query(""" select * from sample_for_labeling""").to_df()

# write df to csv
sample["primary_author_name"].to_csv("validation/names.csv", index=False)
sample.to_csv("validation/full_sample.csv", index=False)